In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from PIL import Image
import os

year = 2002

file_names = [f'./{year}cme/univ{year}_{"%02d" % month}.txt' for month in range(1, 13)]

cme_all = pd.DataFrame()
for file in file_names:
    cme_month = pd.read_fwf(
        file,
        skiprows=4,
        header=None
    )
    cme_all = pd.concat([cme_all, cme_month])

is_Halo = cme_all.iloc[:, 12].astype(str).str.contains("Halo", na=False)

cme = cme_all.iloc[:, 10].astype(str).str[:3]
cme = pd.to_numeric(cme, errors='coerce')

cme[cme.isna() | ~is_Halo] = 0
cme = pd.to_numeric(cme, errors='coerce')

cme_event = pd.DataFrame({
    "date": cme_all.iloc[:, 0],
    "strength": cme
})
cme_event

# 1) 먼저 date를 datetime으로 변환
cme_event["date"] = pd.to_datetime(cme_event["date"])

# 2) 그 다음에 일별 합치기
cme_daily = cme_event.groupby("date", as_index=False)["strength"].sum()

# 3) 날짜 range 만들기 (datetime 기준)
full_dates = pd.date_range(cme_daily["date"].min(), cme_daily["date"].max(), freq="D")

# 4) reindex로 빠진 날짜 채우기
cme_filled = (
    cme_daily
    .set_index("date")
    .reindex(full_dates, fill_value=0)
    .rename_axis("date")
    .reset_index()
)

# 1) 먼저 date를 datetime으로 변환
cme_event["date"] = pd.to_datetime(cme_event["date"])

# 2) 그 다음에 일별 합치기
cme_daily = cme_event.groupby("date", as_index=False)["strength"].sum()

# 3) 날짜 range 만들기 (datetime 기준)
full_dates = pd.date_range(cme_daily["date"].min(), cme_daily["date"].max(), freq="D")

# 4) reindex로 빠진 날짜 채우기
cme_filled = (
    cme_daily
    .set_index("date")
    .reindex(full_dates, fill_value=0)
    .rename_axis("date")
    .reset_index()
)

In [2]:
pickle.dump(cme_filled['strength'].values, open(f"cme_{year}.pkl", "wb"))

In [3]:
cme_filled

,date,strength
0,2002-01-01,0.0
1,2002-01-02,0.0
2,2002-01-03,0.0
3,2002-01-04,0.0
4,2002-01-05,0.0
...,...,...
360,2002-12-27,0.0
361,2002-12-28,0.0
362,2002-12-29,0.0
363,2002-12-30,0.0
